### Explore the Telco dataset 
- [x] Pull in data
- [x] Prep data (including splitting it)
- [ ] Form more specific questions to ask of the data
- [ ] Explore univariate data
- [ ] Explore bivariate data (to answer questions)
- [ ] Explore multivariate data (optional)

In [8]:
# import libraries
import acquire
import prepare
import pandas as pd
import numpy as np

In [16]:
acquire.print_libs()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from pydataset import data
import os
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics
import warnings
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


In [50]:
df = acquire.get_telco_data()

Reading from file...


In [52]:
acquire.df_info(df)

,nunique,dtypes,isnull,59
payment_type_id,4,int64,0,3
internet_service_type_id,3,int64,0,2
contract_type_id,3,int64,0,2
customer_id,7043,object,0,0103-CSITQ
gender,2,object,0,Female
senior_citizen,2,int64,0,0
partner,2,object,0,Yes
dependents,2,object,0,No
tenure,73,int64,0,57
phone_service,2,object,0,Yes


In [54]:
df = prepare.prep_telco(df)

In [66]:
acquire.df_info(df,samples=1)

,nunique,dtypes,isnull,6394-HHHZM
gender,2,object,0,Male
senior_citizen,2,object,0,No
married,2,object,0,No
children,2,object,0,No
tenure_months,73,int64,0,3
paperless_billing,2,object,0,No
monthly_charges,1585,float64,0,20.7
total_charges,6531,float64,0,57.5
churn,2,object,0,No
contract_type,3,object,0,Month-to-month


In [74]:
tvt_set = train,validate,test = prepare.split_df(df,'churn')

In [120]:
# Verify the dataframe split correctly
for df in tvt_set:
    print(df.shape)

(4225, 16)
(1409, 16)
(1409, 16)


> Let's start asking some questions.
- Does internet (or lack of) affect churn? If so, do any of the internet services have an impact on churn?
- What is the relationship between monthly charges and churn?
- What contract types cause higher churn?
- (OPT) How do the services affect monthly charges?
- (OPT) Is there any correlation between family life(being married/having kids) and churn?
- (OPT) Does tenure have an impact on churn? (think count of churn against tenure for linear progression)

In [82]:
# View train
acquire.df_info(train)

,nunique,dtypes,isnull,0742-NXBGR
gender,2,object,0,Female
senior_citizen,2,object,0,No
married,2,object,0,No
children,2,object,0,No
tenure_months,73,int64,0,1
paperless_billing,2,object,0,Yes
monthly_charges,1374,float64,0,82.3
total_charges,4006,float64,0,82.3
churn,2,object,0,Yes
contract_type,3,object,0,Month-to-month


In [118]:
# play with checking out dropping columns
acquire.df_info(prepare.drop_extras(train,9))

,nunique,dtypes,isnull,7599-FKVXZ
children,2,object,0,No
tenure_months,73,int64,0,42
paperless_billing,2,object,0,Yes
monthly_charges,1374,float64,0,106.15
total_charges,4006,float64,0,4512.7
churn,2,object,0,Yes
contract_type,3,object,0,Month-to-month
internet_service_type,3,object,0,Fiber optic
payment_type,4,object,0,Bank transfer (automatic)
streaming,5,object,0,Both


In [115]:
# get obj_cols
obj_col = []

for col in train.columns:
    # print(col)
    if train[col].dtype == 'O':
        # print('object')
        obj_col.append(col)

obj_col

['gender',
 'senior_citizen',
 'married',
 'children',
 'paperless_billing',
 'churn',
 'contract_type',
 'internet_service_type',
 'payment_type',
 'streaming',
 'phone_lines',
 'protection',
 'support']

In [ ]:
train.

### Build the encoders
- [ ] Drop columns for the models (build function if necessary)
- [ ] Encode the data
- [ ] 